# Stargunt II

Let's take a look at combat odds in Stargrunt II.

(If you aren't a programmer, or just want to, you can ignore things like the block below; that is Python code to calculate and format the probabilities.)

In [1]:
import scipy
from scipy.stats import binom
from prettytable import PrettyTable

We need some functions:

- prob: Probability of rolling a given value on a single d-sided dice.
- prob_gt: Probability of rolling greater than v on a single d-sided dice.
- prob_opposed: Probability of offensive win, given defense rolling d-sided die and offense rolling o-sided die.
  See opposed die rolls below. `factor` is a multiplier of the offensive value (see [Wounds and kills below](http://localhost:8889/notebooks/Stargrunt%20II.ipynb#Wounds-and-kills)).
- prod: Like sum(), but more multiplier.
- by_one: Iterate over the list os, yielding a pair of each element and the remaining list.
- inv: The probability of not-p.
- pct: Return a relatively pretty percentage string.

In [2]:
def prob(d):
    return 1 / d

def prob_gt(d, v):
    if d > v:
        return (d - v) * prob(d)
    else:
        return 0.0

def prob_opposed(d, o, factor=1):
    sum = 0.0
    for i in range(1, d + 1):
        sum += prob(d) * prob_gt(o, i * factor)
    return sum

def prod(vs):
    return reduce(operator.mul, vs, 1.0)

def by_one(os):
    for i in range(len(os)):
        yield (os[i], os[0:i] + os[i+1:])

def inv(p):
    return 1 - p

def pct(p):
    return f'{p*100:2.3}%'

## Opposed die rolls

Stargrunt uses an "opposed die roll" mechanism; one player, the defense, rolls one die and the other player, the offense, rolls one or more dice, counting the number of successes where an offensive die is greater than the defensive die. The game uses the following dice to adjust the difficulty of the roll: 4-sided (D4), 6-sided (D6), 8-sided (D8), 10-sided (D10), 12-sided (D12). In the following table, the defensive die is on the vertical axis, the offensive die is on the horizontal axis.

In [3]:
DiceRange = [4, 6, 8, 10, 12]
M = PrettyTable()
M.field_names = [''] + ['D%s' % d for d in DiceRange]
for d in DiceRange:
    M.add_row([f"D{d}"] + [f"{pct(prob_opposed(d,o))}" for o in DiceRange])
print(M)

+-----+-------+-------+-------+-------+-------+
|     |   D4  |   D6  |   D8  |  D10  |  D12  |
+-----+-------+-------+-------+-------+-------+
|  D4 | 37.5% | 58.3% | 68.8% | 75.0% | 79.2% |
|  D6 | 25.0% | 41.7% | 56.2% | 65.0% | 70.8% |
|  D8 | 18.8% | 31.2% | 43.8% | 55.0% | 62.5% |
| D10 | 15.0% | 25.0% | 35.0% | 45.0% | 54.2% |
| D12 | 12.5% | 20.8% | 29.2% | 37.5% | 45.8% |
+-----+-------+-------+-------+-------+-------+


For example, rolling a D6 against a D8 yields a 31.2% chance of success. Likewise, a D10 against a D4 has a 75% chance of success.

## Multiple oppossed rolls

You will notice that it's not symmetrical: a D4 rolling against another D4 only has a 37.5% chance of success. This is because ties are broken in favor of the defensive roll: a 2 rolled against another 2 is not a success.

The fire procedure in Stargrunt has the defender rolling one die and the attacker rolling two or more. The outcome of the roll is 

- Failure, if none of the rolls succeed,
- Minor success, if one of the rolls succeds, or
- Major success, if two or more of the rolls succeed.

In [4]:
from functools import reduce
import operator

def prob_failure(d, os):
    if len(os) == 0: return 0.0
    return prod([inv(prob_opposed(d, o)) for o in os])

def prob_minor(d, os):
    if len(os) == 0: return 0.0
    return sum([
        prob_opposed(d, o1) * prod(map(lambda x: inv(prob_opposed(d, x)), os2))
        for (o1, os2) in by_one(os)
    ])

def prob_major(d, os):
    if len(os) == 0: return 0.0
    sum = 0.0
    f   = 1.0
    for i in range(len(os)):
        g = 1.0
        for j in range(i + 1, len(os)):
            sum += f * g * prob_opposed(d, os[i]) * prob_opposed(d, os[j])
            g   *= 1 - prob_opposed(d, os[j])
        f *= inv(prob_opposed(d, os[i]))
    return sum

(prob_failure(4,[6,8,10,12]) + prob_minor(4,[6,8,10,12]) + prob_major(4,[6,8,10,12]))

1.0

And yay! The probabilities of the three possibiliies add up to 1.0.

### A firing example

There are a number of details needed to work out the results of fire combat.

**Unit quality, coupled with a die type and range band width.**

| Unit Quality | Die | Range band |
| ---          | --- | ---------- |
| Untrained | D4 | 4'' |
| Green     | D6 | 6'' |
| Regular   | D8 | 8'' |
| Veteran   | D10 | 10'' |
| Elite     | D12 | 12'' |

The range band is used to provide the defensive die on an opposed roll: A target within the first range band has a range die of D4, each farther range band shifts up one die type. Also, cover and a few other factors increase the range die.

**Armor type and armor die.**

| Armor type | Armor die |
| ---------- | --------- |
| Basic battledress | D4 |
| Partial light armor | D6 |
| Full light armor | D8 |
| Combat power suit / light powered armor | D10 |
| Heavy power armor | D12 |

The Armor die of a target is used as the defensive die in an opposed roll to determine the damage resulting from hits.

**Normal squad small arms, with firepower value (used to determine hits) and impact value (to determine damage).**

| Weapon | Firepower value | Impact value |
| ------ | --------------- | ------------ |
| Hunting Rifle | 1 | D10 |
| Assault Rifle, low-tech | 2 | D8 |
| Assault Rifle, low-tech, grenade launcher | 3 | D8 |
| Assault Rifle, advanced | 2 | D10 |
| Assault Rifle, advanced, grenade launcher | 3 | D10 |
| Gauss Rifle | 2 | D12 |
| Gauss Rifle, grenade launcher | 3 | D12 |

The firepower value of a squad are multiplied by the number of soldiers carrying them, rounded up to the next die type, with a maximum die type of D12. The impact value is used as the offensive die in the damage roll.

**Squad support weapns, with support firepower and impact value.**

| Weapon | Firepower die | Impact value |
| ------ | --------------- | ------------ |
| Conventional machine gun (SAW) | D8 | D10 |
| Rotary machine gun (SAW) | D10 | D10 |
| Gauss machine gun (SAW) | D10 | D12 |
| Infantry plasma gun | D6 | D12* |
| Automatic grenade launcher | D12 | D8* |
| Multiple rocket launcher pack (MLP) | D8 | D8* |
| Infantry anti-vehicle rocket | D10 | D12* |

The firepower die are used as part of the multiple opposed roll to determine initial hits.

Starred impact values indicate anti-vehicle use.


**Example 1**

A regular (D8) squad of 8  soldiers consisting of 7 advanced assault rifles (In Stargrunt, the number of small arms (7) is multiplied by a firepower factor (2) and the result rounded up to the next die type, limited to D12.) and the squad's Gauss machine gun (firepower of D10) fires on another squad wearing partial light armor (D6), behind soft cover, at a distance of about 140 meters (14'' ground scale, range band 2 for regular troops, up one for the cover: range die of D8).

In [5]:
ex1_pf = prob_failure(8, [8,12,10])
ex1_pmi = prob_minor(8, [8,12,10])
ex1_pma = prob_major(8, [8,12,10])
print(f'{pct(ex1_pf)} failure, {pct(ex1_pmi)} minor success, and {pct(ex1_pma)} major success')

9.49% failure, 34.8% minor success, and 55.7% major success


The results are a 9.5% chance of a failure, resulting in no hits, a 34.8% chance of a minor success, resulting in the targeted squad being suppressed, and a 55.7% chance of a major success, resulting in suppression and possible casualties.

**Example 2**

If the same squad attacks without using the Gauss machine gun, and at 400 meters, the results are different.

In [6]:
ex2_pf = prob_failure(12,[8,12])
ex2_pmi = prob_minor(12,[8,12])
ex2_pma = prob_major(12,[8,12])
print(f'{pct(ex2_pf)} failure, {pct(ex2_pmi)} minor success, and {pct(ex2_pma)} major success')

38.4% failure, 48.3% minor success, and 13.4% major success


In this case, a minor success (and suppression) is most likely and there is a higher probability of a failure.

### Suppression

A squad which is suppressed cannot take most actions until its suppression is relieved. (In addition, squad may be suppressed multiple times, which must be relieved one at a time.) Removing suppression requires the squad to roll the unit quality die (D8 in the examples above) greater than the squad leader's leadership value.

In [18]:
M = PrettyTable()
M.field_names = [
    'Leadership', 'Untrained', 'Green', 'Regular', 'Vetran', 'Elite'
]
M.add_row(['LV1 (Good)'] + [f'{pct(prob_gt(d, 1))}' for d in DiceRange])
M.add_row(['LV2 (Avg)']  + [f'{pct(prob_gt(d, 2))}' for d in DiceRange])
M.add_row(['LV3 (Poor)'] + [f'{pct(prob_gt(d, 3))}' for d in DiceRange])
print(M)

+------------+-----------+-------+---------+--------+-------+
| Leadership | Untrained | Green | Regular | Vetran | Elite |
+------------+-----------+-------+---------+--------+-------+
| LV1 (Good) |   75.0%   | 83.3% |  87.5%  | 90.0%  | 91.7% |
| LV2 (Avg)  |   50.0%   | 66.7% |  75.0%  | 80.0%  | 83.3% |
| LV3 (Poor) |   25.0%   | 50.0% |  62.5%  | 70.0%  | 75.0% |
+------------+-----------+-------+---------+--------+-------+


In the examples, I'll assume the targeted squad is also regular (D8) quality with 8 soldiers and average (LV2) leadership. Therefore, the squad as a 75% chance of removing suppression.

### Potential hits

In the case of a major success, the number of potential hits is calculated by totaling the offensive dice and dividing by the range die type. To calculate the total, I will use the expected value of the given dice. This is given by the average of the values on the die.

In [8]:
def expected(d):
    return sum(range(1, d+1)) / d

def potential_hits(rd, os):
    return sum([expected(o) for o in os]) / rd

| Die  | Expected value |
| ---  | -------------- |
| D4  | {{ expected(4) }} |
| D6  | {{ expected(6) }} |
| D8  | {{ expected(8) }} |
| D10  | {{ expected(10) }} |
| D12  | {{ expected(12) }} |

**Example 1 (cont'd)**

Assuming the squad rolls a major success, the potential hits are potential_hits(8,[8,12,10]), {{potential_hits(8,[8,12,10])}}.

**Example 2 (cont'd)**

In this case, if the squad rolls a major success, the potential hits are {{potential_hits(12,[8,12])}}.

### Wounds and kills

Each potential hit is then treated one at a time (a partial hit, the result of the division, is handled by another roll to determine if you add the extra hit or not), with an opposed roll between the target's armor die versus the firer's impact die. If the impact roll is greater than the armor roll, a wound is sustained; if the impact roll is greater than twice the armor roll, the target is killed outright.

The table below shows the probabilities of a wound/kill given the armor and impact dice.

In [9]:
def p_kill(a,d):
    return prob_opposed(a,d,2)

def p_wound(a,d):
    return prob_opposed(a,d) - p_kill(a,d)

M = PrettyTable()
M.field_names = ['Armor'] + [f'Impact D{d}' for d in DiceRange]
for a in DiceRange:
    M.add_row(
        [f'D{a}'] +
        [f'{pct(p_wound(a,d))}/{pct(p_kill(a,d))}' for d in DiceRange])
print(M)

wound = p_wound(6,10)
kill  = p_kill(6,10)
uninj = inv(wound + kill)

+-------+-------------+-------------+-------------+-------------+-------------+
| Armor |  Impact D4  |  Impact D6  |  Impact D8  |  Impact D10 |  Impact D12 |
+-------+-------------+-------------+-------------+-------------+-------------+
|   D4  | 25.0%/12.5% | 33.3%/25.0% | 31.2%/37.5% | 25.0%/50.0% | 20.8%/58.3% |
|   D6  | 16.7%/8.33% | 25.0%/16.7% | 31.2%/25.0% | 31.7%/33.3% | 29.2%/41.7% |
|   D8  | 12.5%/6.25% | 18.8%/12.5% | 25.0%/18.8% | 30.0%/25.0% | 31.2%/31.2% |
|  D10  |  10.0%/5.0% | 15.0%/10.0% | 20.0%/15.0% | 25.0%/20.0% | 29.2%/25.0% |
|  D12  | 8.33%/4.17% | 12.5%/8.33% | 16.7%/12.5% | 20.8%/16.7% | 25.0%/20.8% |
+-------+-------------+-------------+-------------+-------------+-------------+


**Example 1 (cont'd)**

There are 2 potential hits. The armor of the target is partial light armor (D6) and the impact of the advanced assualt rifle is D10, with the result of a {{pct(wound)}} chance of a wound and a {{pct(kill)}} chance of a kill. (Weirdly, the impact value of the support weapon is not used here.)

| Wounds | Kills | Probability |
| ------ | ----- | ----------- |
| 0 | 0 | {{ pct( pow(kill,2) ) }} |
| 1 | 0 | {{ pct( 2 * wound * uninj ) }} |
| 2 | 0 | {{ pct( pow(wound, 2) ) }} |
| 0 | 1 | {{ pct( 2 * uninj * kill ) }} |
| 0 | 2 | {{ pct( pow(kill,2) ) }} |
| 1 | 1 | {{ pct( 2 * wound * kill ) }} |

**Example 2 (cont'd)**

There is one potential hit, with a {{pct(wound)}} chance of wounding and {{pct(kill)}} chance of killing the target.

### But wait, there's more

A squad can reorganize, among other things performing first aid on wounded members. When doing so, the player rolls a D6.

| Roll | Result |
| ---- | ------ |
| 1-2  | Death  |
| 3-5  | Stable |
| 6    | Ok     |

Yes, there's a 2/6 chance that a soldier not killed outright can still die, and a 1/6 chance that the soldier can return to combat.

### Wrapping up fire combat

To wrap up, `fire_results` shows a summary of the results of one squad firing upon an other: the probability that the target is suppressed, the expected number of wounded, but stable, soldiers, and the expected nmuber of KIAs.

- `prob_suppresed` is the probability of supression of the target.
- `prob_hits` is the probability of one or more hits on the target.
- `expected_hits` is the expected number of hits on the target (i.e., the probability of hits multiplied by the expected number of hits given the dice).
- `expected_casualties` returns the expected number of targets wounded and killed (as a pair), given the dice. It includes the effects of fractional expected hits and the first aid roll.
- `fire_results` prints out `prob_suppressed` and `expected_casualties` somewhat prettily.

In [13]:
def prob_suppressed(rangeD, firepowerDs):
    return prob_minor(rangeD, firepowerDs) + prob_major(rangeD, firepowerDs)

def prob_hits(rangeD, firepowerDs):
    return prob_major(rangeD, firepowerDs)

def expected_hits(rangeD, firepowerDs):
    return prob_hits(rangeD, firepowerDs) * potential_hits(rangeD, firepowerDs)

def expected_casualties(rangeD, armorD, firepowerDs, impactD):
    eKills = 0.0
    eWounds = 0.0
    subTerm = 1.0
    eHits = expected_hits(rangeD, firepowerDs)
    while eHits > 0:
        if eHits < subTerm:
            subTerm = eHits
        eKills  += subTerm * p_kill(armorD, impactD)
        eWounds += subTerm * p_wound(armorD, impactD)
        eHits -= subTerm
    # First aid for wounds
    eKills += (2.0/6.0) * eWounds
    eWounds = (3.0/6.0) * eWounds
    return (eWounds, eKills)

def fire_results(rangeD, armorD, firepowerDs, impactD):
    print(f'{pct(prob_suppressed(rangeD, firepowerDs))} suppression')
    (eWound,eKill) = expected_casualties(rangeD, armorD, firepowerDs, impactD)
    print(f'{eWound:1.2} expected wounds')
    print(f'{eKill:1.2} expected kills')



90.5% suppression
0.18 expected wounds
0.5 expected kills


**Example 1, (cont'd)**

A regular (D8) squad of 8  soldiers consisting of 7 advanced assault rifles (In Stargrunt, the number of small arms (7) is multiplied by a firepower factor (2) and the result rounded up to the next die type, limited to D12. The impact dice for advanced assualt rifles is D10.) and the squad's Gauss machine gun (firepower of D10) fires on another squad wearing partial light armor (D6), behind soft cover, at a distance of about 140 meters (14'' ground scale, range band 2 for regular troops, up one for the cover: range die of D8).

In [17]:
fire_results(8, 6, [8,12,10], 10)

90.5% suppression
0.18 expected wounds
0.5 expected kills


The squad can expect the target to be suppressed 90% of the time, with one wound every five turns and one kill every two turns.

**Example2 (cont'd)**

The same squad attacks the same target without using the Gauss machine gun, and at 400 meters.

In [11]:
fire_results(12, 6, [8, 12], 10)

61.6% suppression
0.019 expected wounds
0.054 expected kills


In this case, the squad can expect the target to be suppressed 62% of the time, with a wound every fifty turns and a kill every twenty turns.

## Conclusion

Infantry fire combat in StarGrunt II is mostly a game of suppression and not especially lethal.

The Jupyter Python 3 notebook that generated this document is available at https://github.com/tmmcguire/The-Napoleonic-Wars-2e/blob/master/Stargrunt%20II.ipynb. PDF generation using `jupyter nbconvert` uses the `pre_pymarkdown.PyMarkdownPreprocessor` preprocessor.